In [1]:
!pip install pandas openpyxl

Defaulting to user installation because normal site-packages is not writeable


In [4]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-

import json
import datetime
import pandas as pd
import requests
import time

# Parámetros
INPUT_JSON     = 'spain/users_españoles.json'
OUTPUT_XLSX    = 'politicos_españoles.xlsx'
REFERENCE_DATE = datetime.date(2025, 4, 20)
SPARQL_URL     = 'https://query.wikidata.org/sparql'
HEADERS        = {
    'Accept': 'application/json',
    'User-Agent': 'TarjetasVisitaScript/1.0 (tu_email@dominio.com)'
}

# Carga de nombres
with open(INPUT_JSON, encoding='utf-8') as f:
    items = json.load(f)
names = [item['name'] for item in items]

def fetch_data_for(name, max_retries=3):
    q = f"""
    SELECT ?dob ?genderLabel ?partyLabel WHERE {{
      {{ ?person rdfs:label "{name}"@el }} 
      UNION 
      {{ ?person rdfs:label "{name}"@en }}
      # Solo humanos:
      ?person wdt:P31 wd:Q5.
      # (Opcional) Solo ciudadanos de Grecia:
      ?person wdt:P27 wd:Q41.
      OPTIONAL {{ ?person wdt:P569 ?dob. }}
      OPTIONAL {{ ?person wdt:P21  ?gender. }}
      OPTIONAL {{ ?person wdt:P102 ?party. }}
      SERVICE wikibase:label {{ bd:serviceParam wikibase:language "es,en". }}
    }}
    """
    for intento in range(1, max_retries+1):
        res = requests.post(SPARQL_URL, data={'query': q}, headers=HEADERS)
        if res.status_code == 200:
            return res.json()['results']['bindings']
        elif res.status_code == 429:
            wait = 2 ** (intento - 1)
            print(f"  ↻ [{name}] 429, reintento en {wait}s…")
            time.sleep(wait)
        else:
            res.raise_for_status()
    print(f"  ✗ [{name}] Falló tras {max_retries} intentos")
    return []

rows = []
total = len(names)
for idx, name in enumerate(names, start=1):
    print(f"[{idx}/{total}] Procesando «{name}»…")
    recs = fetch_data_for(name)
    time.sleep(1)

    if recs:
        rec = recs[0]
        # Fecha de nacimiento → edad
        dob_str = rec.get('dob', {}).get('value')
        dob = datetime.datetime.fromisoformat(dob_str).date() if dob_str else None
        if dob:
            age = (
                REFERENCE_DATE.year - dob.year
                - ((REFERENCE_DATE.month, REFERENCE_DATE.day) < (dob.month, dob.day))
            )
        else:
            age = None

        # Género: primero mostramos raw, luego lo mapeamos
        g_raw = rec.get('genderLabel', {}).get('value', '')
        g_text = g_raw.lower()
        print(f"    → genderLabel crudo: {g_raw}")
        if any(k in g_text for k in ['male', 'masculino', 'hombre']):
            gender = 'masculino'
        elif any(k in g_text for k in ['female', 'femenino', 'mujer']):
            gender = 'femenino'
        else:
            gender = ''

        # Partido
        party = rec.get('partyLabel', {}).get('value', '')

        print(f"  ✓ edad: {age or 'N/A'}, género: {gender or 'N/A'}, partido: {party or 'N/A'}")
    else:
        age = None
        gender = ''
        party = ''
        print("  – Sin datos obtenidos")

    rows.append({
        'name':   name,
        'age':    age,
        'gender': gender,
        'party':  party
    })

# Exportar a Excel
df = pd.DataFrame(rows)
df.to_excel(OUTPUT_XLSX, index=False)
print(f"\n✅ Archivo generado: {OUTPUT_XLSX}")


[1/172] Procesando «Gloria Elizo»…
  – Sin datos obtenidos
[2/172] Procesando «Andrés Lorite»…
  – Sin datos obtenidos
[3/172] Procesando «Inés Sabanés»…
  – Sin datos obtenidos
[4/172] Procesando «José Alcaraz Martos»…
  – Sin datos obtenidos
[5/172] Procesando «Marisa Saavedra»…
  – Sin datos obtenidos
[6/172] Procesando «Daniel Senderos/»…
  – Sin datos obtenidos
[7/172] Procesando «Jaume Asens»…
  – Sin datos obtenidos
[8/172] Procesando «Marta Martín Llaguno»…
  – Sin datos obtenidos
[9/172] Procesando «Diego Gago Bugarín»…
  – Sin datos obtenidos
[10/172] Procesando «Miguel Gutiérrez»…
  – Sin datos obtenidos
[11/172] Procesando «Rosa Romero»…
  – Sin datos obtenidos
[12/172] Procesando «Juan Luis Soto Buril»…
  – Sin datos obtenidos
[13/172] Procesando «Pilar Cancela/»…
  – Sin datos obtenidos
[14/172] Procesando «Néstor Rego»…
  – Sin datos obtenidos
[15/172] Procesando «Pedro Sánchez»…
  – Sin datos obtenidos
[16/172] Procesando «Laura Borràs»…
  – Sin datos obtenidos
[17/172]

HTTPError: 400 Client Error: Bad Request for url: https://query.wikidata.org/sparql

In [5]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-

import json
import datetime
import pandas as pd
import requests
import time

# Parámetros
INPUT_JSON     = 'spain/users_españoles.json'
OUTPUT_XLSX    = 'politicos_españoles.xlsx'
REFERENCE_DATE = datetime.date(2025, 4, 20)
SPARQL_URL     = 'https://query.wikidata.org/sparql'
HEADERS        = {
    'Accept': 'application/json',
    'User-Agent': 'TarjetasVisitaScript/1.0 (tu_email@dominio.com)'
}

# Carga de nombres
with open(INPUT_JSON, encoding='utf-8') as f:
    items = json.load(f)
names = [item['name'] for item in items]

def fetch_data_for(name, max_retries=3):
    q = f"""
    SELECT ?dob ?genderLabel ?partyLabel WHERE {{
      # Etiqueta en español o inglés
      {{ ?person rdfs:label "{name}"@es }} 
      UNION 
      {{ ?person rdfs:label "{name}"@en }}
      # Solo humanos:
      ?person wdt:P31 wd:Q5.
      # (Opcional) Solo ciudadanos de España:
      ?person wdt:P27 wd:Q29.
      OPTIONAL {{ ?person wdt:P569 ?dob. }}
      OPTIONAL {{ ?person wdt:P21  ?gender. }}
      OPTIONAL {{ ?person wdt:P102 ?party. }}
      SERVICE wikibase:label {{ bd:serviceParam wikibase:language "es,en". }}
    }}
    """
    for intento in range(1, max_retries+1):
        res = requests.post(SPARQL_URL, data={'query': q}, headers=HEADERS)
        if res.status_code == 200:
            return res.json()['results']['bindings']
        elif res.status_code == 429:
            wait = 2 ** (intento - 1)
            print(f"  ↻ [{name}] 429, reintento en {wait}s…")
            time.sleep(wait)
        else:
            res.raise_for_status()
    print(f"  ✗ [{name}] Falló tras {max_retries} intentos")
    return []

rows = []
total = len(names)
for idx, name in enumerate(names, start=1):
    print(f"[{idx}/{total}] Procesando «{name}»…")
    recs = fetch_data_for(name)
    time.sleep(1)

    if recs:
        rec = recs[0]
        # Fecha de nacimiento → edad
        dob_str = rec.get('dob', {}).get('value')
        if dob_str:
            dob = datetime.datetime.fromisoformat(dob_str).date()
            age = (REFERENCE_DATE.year - dob.year
                   - ((REFERENCE_DATE.month, REFERENCE_DATE.day) < (dob.month, dob.day)))
        else:
            age = None

        # Género
        g_raw = rec.get('genderLabel', {}).get('value', '')
        g_text = g_raw.lower()
        if any(k in g_text for k in ['male', 'masculino', 'hombre']):
            gender = 'masculino'
        elif any(k in g_text for k in ['female', 'femenino', 'mujer']):
            gender = 'femenino'
        else:
            gender = ''

        # Partido
        party = rec.get('partyLabel', {}).get('value', '')

        print(f"  ✓ edad: {age or 'N/A'}, género: {gender or 'N/A'}, partido: {party or 'N/A'}")
    else:
        age = None
        gender = ''
        party = ''
        print("  – Sin datos obtenidos")

    rows.append({
        'name':   name,
        'age':    age,
        'gender': gender,
        'party':  party
    })

# Exportar a Excel
df = pd.DataFrame(rows)
df.to_excel(OUTPUT_XLSX, index=False)
print(f"\n✅ Archivo generado: {OUTPUT_XLSX}")


[1/172] Procesando «Gloria Elizo»…
  ✓ edad: 58, género: femenino, partido: Podemos
[2/172] Procesando «Andrés Lorite»…
  – Sin datos obtenidos
[3/172] Procesando «Inés Sabanés»…
  ✓ edad: 71, género: femenino, partido: Izquierda Unida
[4/172] Procesando «José Alcaraz Martos»…
  – Sin datos obtenidos
[5/172] Procesando «Marisa Saavedra»…
  – Sin datos obtenidos
[6/172] Procesando «Daniel Senderos/»…
  – Sin datos obtenidos
[7/172] Procesando «Jaume Asens»…
  – Sin datos obtenidos
[8/172] Procesando «Marta Martín Llaguno»…
  ✓ edad: 52, género: femenino, partido: Ciudadanos
[9/172] Procesando «Diego Gago Bugarín»…
  – Sin datos obtenidos
[10/172] Procesando «Miguel Gutiérrez»…
  ✓ edad: 23, género: masculino, partido: N/A
[11/172] Procesando «Rosa Romero»…
  ✓ edad: 55, género: femenino, partido: Partido Popular
[12/172] Procesando «Juan Luis Soto Buril»…
  – Sin datos obtenidos
[13/172] Procesando «Pilar Cancela/»…
  – Sin datos obtenidos
[14/172] Procesando «Néstor Rego»…
  ✓ edad: 63

In [11]:
import pandas as pd
import json
import math

# Rutas de archivos
excel_path = "politicos_españoles.xlsx"  # Ajusta al nombre de tu archivo
json_path  = "politicos_españoles.json"

# 1) Leer el Excel en un DataFrame
df = pd.read_excel(excel_path)

# 2) Convertir a lista de registros (diccionarios)
records = df.to_dict(orient="records")

# 3) Sustituir any NaN --> None para JSON válido
for rec in records:
    for k, v in rec.items():
        if isinstance(v, float) and math.isnan(v):
            rec[k] = None

# 4) Guardar como JSON
with open(json_path, "w", encoding="utf-8") as f:
    json.dump(records, f, ensure_ascii=False, indent=2)

print(f"✅ Convertido '{excel_path}' ({len(records)} filas) a '{json_path}'.")


✅ Convertido 'politicos_españoles.xlsx' (172 filas) a 'politicos_españoles.json'.


In [ ]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-

import json

# Rutas a tus archivos (ajusta si hace falta)
POLITICOS_IN  = 'politicos_españoles.json'
USERS_IN      = 'spain/users_españoles.json'
POLITICOS_OUT = 'politicos_españoles_actualizado.json'

# 1. Carga de los datos
with open(POLITICOS_IN, encoding='utf-8') as f:
    politicos = json.load(f)

with open(USERS_IN, encoding='utf-8') as f:
    users = json.load(f)

# 2. Construcción del diccionario name → party
party_map = {u['name']: u['party'] for u in users}

# 3. Actualización de cada entrada
for entry in politicos:
    if entry['name'] in party_map:
        entry['party'] = party_map[entry['name']]

# 4. Guardar el JSON actualizado
with open(POLITICOS_OUT, 'w', encoding='utf-8') as f:
    json.dump(politicos, f, ensure_ascii=False, indent=2)

print(f"✅ Partidos actualizados y guardados en «{POLITICOS_OUT}»")


In [16]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-

import json
import datetime
import requests
import time
import pandas as pd

# — Configuración de rutas —
# Ajusta INPUT_JSON al nombre de tu JSON de entrada con name, gender y party ya completos.
INPUT_JSON     = 'politicos_españoles_actualizado.json'
OUTPUT_JSON    = 'politicos_españoles_con_edad.json'
OUTPUT_XLSX    = 'politicos_españoles_con_edad.xlsx'

# — Parámetros de cálculo —
REFERENCE_DATE = datetime.date(2025, 4, 20)
SPARQL_URL     = 'https://query.wikidata.org/sparql'
HEADERS        = {
    'Accept': 'application/json',
    'User-Agent': 'TarjetasVisitaScript/1.0 (tu_email@dominio.com)'
}
MAX_RETRIES    = 3
PAUSE_SECONDS  = 1  # pausa entre consultas para no saturar el endpoint

def fetch_dob(name: str) -> str | None:
    """
    Consulta Wikidata para obtener la fecha de nacimiento (ISO YYYY‑MM‑DD)
    de la entidad con la etiqueta `name` en es o en.
    Devuelve el string ISO o None si no hay dob.
    """
    query = f"""
    SELECT ?dob WHERE {{
      {{ ?person rdfs:label "{name}"@es }} UNION {{ ?person rdfs:label "{name}"@en }}
      ?person wdt:P31 wd:Q5.           # solo humanos
      OPTIONAL {{ ?person wdt:P569 ?dob. }}
    }}
    """
    for intento in range(1, MAX_RETRIES + 1):
        resp = requests.post(SPARQL_URL, data={'query': query}, headers=HEADERS)
        if resp.status_code == 200:
            results = resp.json()['results']['bindings']
            if results and 'dob' in results[0]:
                return results[0]['dob']['value']
            return None
        elif resp.status_code == 429:
            # Espera exponencial antes de reintentar
            time.sleep(2 ** (intento - 1))
        else:
            resp.raise_for_status()
    return None

def main():
    # 1. Carga del JSON existente
    with open(INPUT_JSON, encoding='utf-8') as f:
        politicos = json.load(f)

    # 2. Para cada político, buscamos dob y recalculamos age
    for i, entry in enumerate(politicos, start=1):
        name = entry.get('name', '').strip()
        print(f"[{i}/{len(politicos)}] Procesando edad de «{name}»…")
        dob_iso = fetch_dob(name)
        time.sleep(PAUSE_SECONDS)

        if dob_iso:
            dob = datetime.datetime.fromisoformat(dob_iso).date()
            age = (
                REFERENCE_DATE.year - dob.year
                - ((REFERENCE_DATE.month, REFERENCE_DATE.day) < (dob.month, dob.day))
            )
            entry['age'] = age
            print(f"  ✓ edad actualizada: {age}")
        else:
            entry['age'] = None
            print("  – no se encontró fecha de nacimiento")

    # 3. Guardar JSON con edad recalculada
    with open(OUTPUT_JSON, 'w', encoding='utf-8') as f:
        json.dump(politicos, f, ensure_ascii=False, indent=2)
    print(f"\n✅ JSON guardado en «{OUTPUT_JSON}»")

    # 4. Exportar a Excel
    df = pd.DataFrame(politicos)
    df.to_excel(OUTPUT_XLSX, index=False)
    print(f"✅ Excel generado: «{OUTPUT_XLSX}»")

if __name__ == '__main__':
    main()


[1/172] Procesando edad de «Gloria Elizo»…
  ✓ edad actualizada: 58
[2/172] Procesando edad de «Andrés Lorite»…
  – no se encontró fecha de nacimiento
[3/172] Procesando edad de «Inés Sabanés»…
  ✓ edad actualizada: 71
[4/172] Procesando edad de «José Alcaraz Martos»…
  – no se encontró fecha de nacimiento
[5/172] Procesando edad de «Marisa Saavedra»…
  – no se encontró fecha de nacimiento
[6/172] Procesando edad de «Daniel Senderos/»…
  – no se encontró fecha de nacimiento
[7/172] Procesando edad de «Jaume Asens»…
  – no se encontró fecha de nacimiento
[8/172] Procesando edad de «Marta Martín Llaguno»…
  ✓ edad actualizada: 52
[9/172] Procesando edad de «Diego Gago Bugarín»…
  – no se encontró fecha de nacimiento
[10/172] Procesando edad de «Miguel Gutiérrez»…
  ✓ edad actualizada: 93
[11/172] Procesando edad de «Rosa Romero»…
  ✓ edad actualizada: 55
[12/172] Procesando edad de «Juan Luis Soto Buril»…
  – no se encontró fecha de nacimiento
[13/172] Procesando edad de «Pilar Cancela/»

In [18]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-

import json
import datetime
import pandas as pd
import requests
import time

# Parámetros
INPUT_JSON     = 'grecia_anotados/users_griegos.json'
OUTPUT_XLSX    = 'politicos_griegos.xlsx'
REFERENCE_DATE = datetime.date(2025, 4, 20)
SPARQL_URL     = 'https://query.wikidata.org/sparql'
HEADERS        = {
    'Accept': 'application/json',
    'User-Agent': 'TarjetasVisitaScript/1.0 (tu_email@dominio.com)'
}

# Carga de nombres
with open(INPUT_JSON, encoding='utf-8') as f:
    items = json.load(f)
names = [item['name'] for item in items]

def fetch_data_for(name, max_retries=3):
    q = f"""
    SELECT ?dob ?genderLabel ?partyLabel WHERE {{
      # Etiqueta en griego o inglés
      {{ ?person rdfs:label "{name}"@el }} 
      UNION 
      {{ ?person rdfs:label "{name}"@en }}
      # Solo humanos:
      ?person wdt:P31 wd:Q5.
      OPTIONAL {{ ?person wdt:P569 ?dob. }}
      OPTIONAL {{ ?person wdt:P21  ?gender. }}
      OPTIONAL {{ ?person wdt:P102 ?party. }}
      SERVICE wikibase:label {{ bd:serviceParam wikibase:language "el,en". }}
    }}
    """
    for intento in range(1, max_retries+1):
        res = requests.post(SPARQL_URL, data={'query': q}, headers=HEADERS)
        if res.status_code == 200:
            return res.json()['results']['bindings']
        elif res.status_code == 429:
            wait = 2 ** (intento - 1)
            print(f"  ↻ [{name}] 429, reintento en {wait}s…")
            time.sleep(wait)
        else:
            res.raise_for_status()
    print(f"  ✗ [{name}] Falló tras {max_retries} intentos")
    return []

rows = []
total = len(names)
for idx, name in enumerate(names, start=1):
    print(f"[{idx}/{total}] Procesando «{name}»…")
    recs = fetch_data_for(name)
    time.sleep(1)

    if recs:
        rec = recs[0]
        # Fecha de nacimiento → edad
        dob_str = rec.get('dob', {}).get('value')
        if dob_str:
            dob = datetime.datetime.fromisoformat(dob_str).date()
            age = (REFERENCE_DATE.year - dob.year
                   - ((REFERENCE_DATE.month, REFERENCE_DATE.day) < (dob.month, dob.day)))
        else:
            age = None

        # Género
        g_raw = rec.get('genderLabel', {}).get('value', '')
        g_text = g_raw.lower()
        if any(k in g_text for k in ['male', 'masculino', 'άνδρας', 'man']):
            gender = 'masculino'
        elif any(k in g_text for k in ['female', 'femenino', 'γυναίκα', 'woman']):
            gender = 'femenino'
        else:
            gender = ''

        # Partido
        party = rec.get('partyLabel', {}).get('value', '')

        print(f"  ✓ edad: {age or 'N/A'}, género: {gender or 'N/A'}, partido: {party or 'N/A'}")
    else:
        age = None
        gender = ''
        party = ''
        print("  – Sin datos obtenidos")

    rows.append({
        'name':   name,
        'age':    age,
        'gender': gender,
        'party':  party
    })

# Exportar a Excel
df = pd.DataFrame(rows)
df.to_excel(OUTPUT_XLSX, index=False)
print(f"\n✅ Archivo generado: {OUTPUT_XLSX}")


[1/142] Procesando «Giannis Plakiotakis»…
  ✓ edad: N/A, género: masculino, partido: N/A
[2/142] Procesando «Κατερίνα Νοτοπούλου»…
  ✓ edad: 36, género: femenino, partido: N/A
[3/142] Procesando «Costas Zachariadis»…
  – Sin datos obtenidos
[4/142] Procesando «Χάρης Μαμουλάκης»…
  – Sin datos obtenidos
[5/142] Procesando «Δημήτρης Τζανακόπουλος»…
  ✓ edad: 42, género: masculino, partido: Συνασπισμός Ριζοσπαστικής Αριστεράς - Προοδευτική Συμμαχία
[6/142] Procesando «Βασίλης Βιλιάρδος»…
  ✓ edad: 70, género: masculino, partido: N/A
[7/142] Procesando «Μανούσος Βολουδάκης»…
  ✓ edad: 141, género: masculino, partido: N/A
[8/142] Procesando «Χρήστος Γιαννούλης»…
  ✓ edad: 56, género: masculino, partido: Συνασπισμός Ριζοσπαστικής Αριστεράς - Προοδευτική Συμμαχία
[9/142] Procesando «andreasxanthos»…
  – Sin datos obtenidos
[10/142] Procesando «Stratos Simopoulos»…
  ✓ edad: 65, género: masculino, partido: N/A
[11/142] Procesando «Θεόφιλος Λεονταρίδης»…
  ✓ edad: 69, género: masculino, partido